<a href="https://colab.research.google.com/github/tbrar/CPSC-501-A4-TensorFlow/blob/master/CHDModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [3]:
from google.colab import files
uploaded = files.upload()

Saving heart_train.csv to heart_train.csv


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [0]:
train_file_path = "heart_train.csv"
test_file_path = "heart_test.csv"

In [11]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

SELECT_COLUMNS = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age', 'chd']
DEFAULTS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

example_batch, labels_batch = next(iter(temp_dataset)) 
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

example_batch, labels_batch = next(iter(temp_dataset)) 



[[142.     2.4    2.55  23.89  54.    26.09  59.14  37.  ]
 [124.     0.16   2.44  16.67  65.    24.58  74.91  23.  ]
 [128.     6.6    3.58  20.71  55.    24.15   0.    52.  ]
 [134.     0.57   4.75  23.07  67.    26.33   0.    37.  ]
 [130.     1.22   3.3   13.65  50.    21.4    3.81  31.  ]]

[0 0 0 0 0]


In [15]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

example_batch, labels_batch = next(iter(packed_train_data)) 

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,139.088384,3.534470,4.721641,25.417854,52.611111,25.980404,17.155328,42.727273
std,20.883153,4.622259,1.969109,7.862184,9.908200,4.185559,24.696052,14.758524
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000
25%,125.500000,0.050000,3.300000,19.825000,46.000000,22.757500,0.510000,31.000000
50%,134.000000,1.800000,4.325000,26.115000,53.000000,25.780000,7.780000,44.500000
75%,148.500000,5.152500,5.632500,31.102500,59.000000,28.402500,23.737500,55.250000
max,218.000000,31.200000,14.160000,42.490000,77.000000,45.720000,147.190000,64.000000


In [16]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()


array([[-0.5309724 , -0.76466286, -1.123169  ,  0.50013405,  0.03924924,
        -0.9390394 , -0.6391843 ,  1.305871  ],
       [ 1.0013632 ,  0.14398377,  1.0351683 ,  1.5990655 ,  0.84666127,
         1.6555964 , -0.5695375 ,  0.76381135],
       [ 1.0492487 ,  1.1824372 , -0.03638268, -1.304708  ,  0.5438818 ,
        -0.5304916 ,  1.0546087 ,  0.22175172],
       [ 0.04365342,  1.0958995 , -0.41726568,  0.857541  , -0.06167727,
         0.60436285, -0.24478929,  1.4413859 ],
       [ 0.23519537, -0.6781251 , -0.04146124,  0.59425545, -2.2820604 ,
         0.33677593, -0.6646944 ,  0.8315688 ]], dtype=float32)

In [17]:
CATEGORIES = {
    'famhist': ['Absent', 'Present'],
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

# See what you just created.
categorical_columns

categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1.]


In [18]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])


[ 0.          1.         -0.5309724  -0.76466286 -1.123169    0.50013405
  0.03924924 -0.9390394  -0.6391843   1.305871  ]


In [19]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

model.fit(train_data, epochs=20)

test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

predictions = model.predict(test_data)

# Show some results
for prediction, chd in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted chd: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("Has CHD" if bool(chd) else "Doesn't have CHD"))


Epoch 1/20
80/80 [==============================] - 2s 19ms/step - loss: 0.5683 - accuracy: 0.7045
Epoch 2/20
80/80 [==============================] - 0s 3ms/step - loss: 0.5154 - accuracy: 0.7424
Epoch 3/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4954 - accuracy: 0.7601
Epoch 4/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4854 - accuracy: 0.7652
Epoch 5/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4674 - accuracy: 0.7601
Epoch 6/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4627 - accuracy: 0.7778
Epoch 7/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4614 - accuracy: 0.7677
Epoch 8/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4410 - accuracy: 0.7778
Epoch 9/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4447 - accuracy: 0.7980
Epoch 10/20
80/80 [==============================] - 0s 3ms/step - loss: 0.4220 - accuracy: 0.7854
Epoch 11/20
80/80 